## Boundary matrix, reduction, output barcode (sparse matrix)

Creating simplex class and reading function

In [1]:
import numpy as np

class simplex:
    def __init__(self, line):
        self.dim, self.val, self.vert = self.parse_line(line)
    
    def parse_line(self, line):
        splited_line = line.split()
        val = float(splited_line[0])
        dim = int(splited_line[1])
        vert = [int(x) for x in splited_line[2:]]
        assert(len(vert)==dim+1)
        return dim, val, vert

def read_filtration(name):
    simplices = []
    with open(name, "r") as ins:
        for line in ins:
            simplices.append(simplex(line))
    return simplices

Reading the example filtration

In [2]:
simplices = read_filtration('filtration.txt')

Grouping by time

In [3]:
simplices.sort(key=lambda x: x.val)

Sorting the vertices

In [4]:
for s in simplices:
    s.vert.sort()

Creating dictionary for matrix creation

In [5]:
simplices_str =  [str(s.vert).strip('[]') for s in simplices]

d = {}
for i, s in enumerate(simplices_str):
    d[s] = i

Creating dictionary for final output

In [6]:
sDict = {}
for i, s in enumerate(simplices):
    sDict[i] = s

Creating sparse matrix

In [7]:
M = len(simplices)

sparse = [[] for i in range(M)]

for i, s in enumerate(simplices):
    if s.dim>0:
        vs = list(s.vert)
        for v in vs:
            temp = list(vs)
            temp.remove(v)
            sparse[i].append(d[str(temp).strip('[]')])
            
for i in range(M):
    sparse[i].sort()

Printing sparse matrix

In [8]:
print sparse

[[], [], [], [0, 1], [0, 2], [], [1, 5], [2, 5], [1, 2], [6, 7, 8]]


Reducing Matrix

In [9]:
lowsparse = [float('inf')]*M
lowdict = {}

def xor(col1, col2):
    c = []
    i, j = 0, 0
    n1, n2 = len(col1), len(col2)
    while(i<n1 and j<n2):
        if(col1[i]<col2[j]):
            c.append(col1[i])
            i+=1
        elif(col1[i]>col2[j]):
            c.append(col2[j])
            j+=1
        else:
            i+=1
            j+=1
    while i<n1:
        c.append(col1[i])
        i+=1
    while j<n2:
        c.append(col2[i])
        j+=1
    return c

for j in range(M): #*O(M)
    if sparse[j]: 
        lowcol = sparse[j][-1]
        lowsparse[j] = lowcol
        if lowcol not in lowdict:
            lowdict[lowcol] = j
            continue
        while lowcol in lowdict: #*O(M) in the worst case
            ind = lowdict[lowcol]
            sparse[j] = xor(sparse[j],sparse[ind]) #*O(M) in the worst case too (depends on the size of the sparse columns)
            if sparse[j]:
                lowcol = sparse[j][-1]
                lowsparse[j] = lowcol
            else:
                lowsparse[j] = float("inf")
                break

## Explaning reduction complexity
As the comments in the code above explained, the three lines of code that actually change the whole code's complexity are commented, the other ones are only O(1), so they don't actually change the complexity of the algorithm. Therefore the complexity is O(M^3).

Printing reduced matrix

In [10]:
print sparse

[[], [], [], [0, 1], [0, 2], [], [1, 5], [], [], [6, 7, 8]]


Giving final answer

In [36]:
finalanswer = []
for j in range(M):
    if lowsparse[j] == float('inf'):
        if j not in lowsparse:
            aux = sDict[j]
            finalanswer.append(str(aux.dim)+' '+str(aux.val)+' '+'inf')
    else:
        aux1 = sDict[lowsparse[j]]
        aux2 = sDict[j]
        finalanswer.append(str(aux1.dim)+' '+str(aux1.val)+' '+str(aux2.val))
print finalanswer

['0 1.0 inf', '0 1.0 2.0', '0 1.0 2.0', '0 3.0 4.0', '1 4.0 inf', '1 5.0 6.0']


Returning as a txt

In [37]:
def output_codebar(barcodelist,name):
    barcodetxt = open(name,"w")
    for code in barcodelist:
        barcodetxt.write(code)
        barcodetxt.write("\n")
    return

output_codebar(finalanswer,"output.txt")